## UNet

In [18]:
from keras.layers import Input, Conv2D, Dropout, BatchNormalization, MaxPooling2D, Concatenate, UpSampling2D
from keras.layers import Reshape, Activation, Permute, Add
from keras.models import Model
import keras

In [124]:
class Nets():
    def __init__(self):
        pass
    
    def downBlock(self, x, activation='relu', 
                  kernel_initializer='he_normal', dropout_rate=0, 
                  weight_decay=0, add=True, pooling = True):
        
        conv = Conv2D(128, (3, 3), activation=activation, padding='same',
                       kernel_initializer=kernel_initializer)(x)
        conv = Dropout(dropout_rate)(conv)
        conv = Conv2D(64, (2, 2), activation=activation, 
                      padding='same',kernel_initializer=kernel_initializer)(conv)
        pool = None
        
        if add:
            conv = self.addLayer(conv,x)
        
        if pooling:
            pool = MaxPooling2D(pool_size=(2, 2))(conv)
        return pool, conv
    
    def upBlock(self,x, h, activation='relu', 
                  kernel_initializer='he_normal', dropout_rate=0.2, weight_decay=0):
        
        up = Concatenate(axis=-1)([UpSampling2D(size=(2, 2))(x), h])
        conv = Conv2D(128, (3, 3), activation=activation, padding='same',
                      kernel_initializer=kernel_initializer)(up)
        conv = Dropout(dropout_rate)(conv)
        conv = Conv2D(64, (2, 2), activation=activation, 
                      padding='same',kernel_initializer=kernel_initializer)(conv)
        
        return conv

    
    def addLayer(self,x1,x2):
        y = Add()([x1, x2])
        return y
    
    def DeepUnet(self, nClasses, depth = 3 , optimizer='RMSprop', final_activation = 'softmax', kernel_initializer='he_normal',
             dropout_rate = 0.2, input_width=360 , input_height=480 , nChannels=3 ):
         
        inputs = Input((input_height, input_width, nChannels))
        x = Conv2D(64, (3, 3), activation='relu', padding='same',
                   kernel_initializer=kernel_initializer)(inputs)
        conv = []
        
        for i in range(depth):
            x, conv0 = self.downBlock(x,dropout_rate=dropout_rate,  kernel_initializer=kernel_initializer)
            conv.append(conv0)
        
        x, conv0 = self.downBlock(x, pooling=False, kernel_initializer=kernel_initializer)
        x = conv0
        
        for i in range(depth):
            x = self.upBlock(x, conv[depth-i-1], dropout_rate=dropout_rate, kernel_initializer=kernel_initializer)
        
        conv0 = Conv2D(nClasses, (1, 1), activation='relu',padding='same',kernel_initializer=kernel_initializer)(x)
        conv0 = Reshape((nClasses,input_height*input_width))(conv0)
        conv0 = Permute((2,1))(conv0)

        conv0 = Activation(final_activation)(conv0)

        model = Model(input=inputs, output=conv0)

        if not optimizer is None:
            model.compile(loss="categorical_crossentropy", optimizer= optimizer , metrics=['accuracy'] )
        return model
    